In [1]:
import pandas as pa
import numpy as np
import random
import csv
import math
with open("D:/Download/iris.csv", newline= "", encoding="ISO-8859-1") as filecsv:  #importo i dati dal file csv
    DATI = pa.read_csv('D:/Download/iris.csv') 
    XComplete = DATI.iloc[:,0:4] 
    yComplete = DATI.iloc[:,4]
X_Numpy = XComplete.to_numpy(float)
num_labels = 3
[m,n] = XComplete.shape
l = list(range(m))
random.shuffle(l)
XMisto = XComplete.iloc[l,:]
yMisto = yComplete.iloc[l]
size_train = math.ceil(m * 0.6)
size_cv = math.ceil(m * 0.2)
X_ts = XMisto.iloc[0:size_train,:]
y_ts = yMisto.iloc[0:size_train]
X_ts_Numpy = X_ts.to_numpy(float)
y_ts_Numpy = y_ts.to_numpy(float)
X_cv = XMisto.iloc[size_train : size_train+size_cv,:]
X_cv_Numpy = X_cv.to_numpy(float)
y_cv = yMisto.iloc[size_train : size_train+size_cv]
y_cv_Numpy = y_cv.to_numpy(float)
x_test = XMisto.iloc[size_train+size_cv:m,:]
X_test_Numpy = x_test.to_numpy(float)
y_test = yMisto.iloc[size_train+size_cv:m]
y_test_Numpy = y_test.to_numpy(float)

def featureNormalize(X):
    X_norm = X
    mu = X.mean(0)
    sigma = X.std(0)                               #implemento la feature normalize
    r,c = X.shape
    for i in range(c):
        X_norm[:,i] = ((X[:,i] - mu[i]) / sigma[i])
    return X_norm

X_ts_norm = featureNormalize(X_ts_Numpy)
X_cv_norm = featureNormalize(X_cv_Numpy)
X_test_norm = featureNormalize(X_test_Numpy)

dimTotClassi = 30;
valoreInferiore = -3.2;
valoreSuperiore = 3.2;
step = (abs(valoreInferiore)+abs(valoreSuperiore))/dimTotClassi
intervallo = np.array(list(np.arange(valoreInferiore, valoreSuperiore+0.1, step)))

def learningBayes(X, y, pattern, classe):
    num_eventi = sum(pattern)
    [m, n] = X.shape
    contatore = 0
    p = np.zeros(num_eventi)
    for i in range(0, n):
        for j in range(0,pattern[i]):
            p[contatore] = (np.sum(X[np.nonzero(np.array(y[y==classe]))]==j-1)+1) / (np.shape(np.array(y[y== classe]))[0]+pattern[i])
            contatore = contatore + 1
    return p

def transform(X, intervallo):
    [m,n] = X.shape
    for i in range(0,m):
        for j in range(0,n):
            for k in range(0, (np.shape(intervallo)[0]-1)):
                if X[i,j] > intervallo[k] and X[i,j] < intervallo[k + 1]:
                    X[i,j] = k
                    
    return X

X_ts_norm = transform(X_ts_norm,intervallo)
X_cv_norm = transform(X_cv_norm, intervallo)
X_test_norm = transform(X_test_norm, intervallo).astype(int)

pattern = np.array([30,30,30,30])
p1 = learningBayes(X_ts_norm, y_ts, pattern, 1)
p2 = learningBayes(X_ts_norm, y_ts, pattern, 2)
p3 = learningBayes(X_ts_norm, y_ts, pattern, 3)

p1Smoth = (np.shape(np.array(yMisto[yMisto== 1]))[0]) / m+3
p2Smoth = (np.shape(np.array(yMisto[yMisto== 2]))[0]) / m+3
p3Smoth = (np.shape(np.array(yMisto[yMisto== 3]))[0]) / m+3

predizione1 = np.zeros(30);
predizione2 = np.zeros(30);
predizione3 = np.zeros(30);

def previsioni(X_test, prob, pattern, p_class):
    contatore = 0
    p = np.zeros(np.shape(X_test)[0])
    for i in range(0, np.shape(pattern)[0]):
        p[i] = prob[contatore + X_test[i]]
        contatore = contatore + pattern[i]
    p_sum = math.exp(np.sum(np.log(np.transpose(p))))
    pred = p_sum * p_class;
    return pred

for j in range(0,30):
    predizione1[j] = previsioni(X_test_norm[j, :], p1, pattern, p1Smoth)
    predizione2[j] = previsioni(X_test_norm[j, :], p2, pattern, p2Smoth)
    predizione3[j] = previsioni(X_test_norm[j, :], p3, pattern, p3Smoth)

predizioni = np.array([np.transpose(predizione1),np.transpose(predizione2),np.transpose(predizione3)])

probClasse1 = (np.amax(predizioni, axis=0) == np.transpose(predizione1)).astype(int) * 1
probClasse2 = (np.amax(predizioni, axis=0) == np.transpose(predizione2)).astype(int) * 2
probClasse3 = (np.amax(predizioni, axis=0) == np.transpose(predizione3)).astype(int) * 3

risultati = np.transpose(np.array([np.transpose(probClasse1) + np.transpose(probClasse2) + np.transpose(probClasse3)]))

#accuracy = np.shape(np.array(y_test_Numpy[y_test_Numpy == risultati])) #/ (np.shape(y_test)[0])
#print(accuracy)
